In [1]:
# Import modules
import numpy as np

# Import functions
from local_global_matrices import *
from assembly_K_matrices import *
from assembly_B_matrices import *
from assembly_f_vectors import *
from RegularSudomainsMesh import RegularSubdomainsMesh
from utils import *

%load_ext autoreload
%autoreload 2

In [2]:
# Import data
d_dat = np.genfromtxt('data/d.dat')
fP_dat = np.genfromtxt('data/fP.dat')
fr_dat = np.genfromtxt('data/fr.dat')
Ks = np.genfromtxt('data/localK.dat')
solution = np.genfromtxt('data/solution.dat')

In [3]:
# Initial data
# Number of subdomains
Nsub_x = 4
Nsub_y = 3

# Number of remaining nodes in each subdomain
Nr_x = 4
Nr_y = 3

# Local remaining and primal indices
rs = np.array([1, 2, 4, 5, 6, 7, 9, 10])
qs = np.array([0, 3, 8, 11])

In [4]:
# Create mesh
mesh = RegularSubdomainsMesh(Nsub_x, Nsub_y, Nr_x, Nr_y)

In [5]:
# Transformation matrices A
# Primal nodes local-global transformation matrices
APq = create_APq_matrices(mesh)

# Remaining nodes local-global transformation matrices
ARr = create_ARr_matrices(mesh)

In [6]:
# Stiffness matrices K
KRR = assembly_KRR_matrix(Ks, ARr, rs, mesh)
KPP = assembly_KPP_matrix(Ks, APq, qs, mesh)
KRP = assembly_KRP_matrix(Ks, APq, ARr, qs, rs, mesh)
KPR = KRP.T

In [7]:
# Remaining nodes
Nr = Nr_x*Nr_y - 4  # Number of local remaining nodes r
NR = Nr*(Nsub_x*Nsub_y)  # Number of total remaining nodes R

# Primal nodes
Nq = 4 # Number of primal nodes in each subdomain 
NP = (Nsub_x + 1)*(Nsub_y + 1) # Total number of primal nodes

# Lambda nodes
NlambdaR = (Nsub_x*(Nr_x - 2)) * (Nsub_y - 1) + (Nsub_y*(Nr_y - 2)) * (Nsub_x - 1)
NlambdaP_Dir = Nsub_y + 1 
NlambdaR_Dir = (Nr_y - 2)*Nsub_y 
Nlambda = NlambdaR + NlambdaR_Dir + NlambdaP_Dir

In [8]:
# Assembly B matrices
BlambdaR = assembly_BR_matrix(mesh, ARr)

# Dirichlet boundary conditions
# Left wall remaining
BlambdaR = assembly_Dirichlet_BR_matrix(mesh, ARr, BlambdaR)

#Left wall primal
BlambdaP = assembly_Dirichlet_BP_matrix(mesh, APq)

In [9]:
# Assembly d vector
d = np.zeros(Nlambda)
d[NlambdaR:]=d_dat

In [10]:
# Assembly f vectors
# fP
fP = fP_dat

# fR
fR = assembly_fR_vector(mesh, fr_dat)

In [11]:
# Matrices pre computation
KRR_inv = np.linalg.inv(KRR)

SPP = KPP - KPR @ KRR_inv @ KRP
SPP_inv = np.linalg.inv(SPP)

fPH = fP - KPR @ KRR_inv @ fR

dH = -d \
    + BlambdaP @ SPP_inv @ fPH \
    + BlambdaR @ KRR_inv @ fR \
    - BlambdaR @ KRR_inv @ KRP @ SPP_inv @ fPH

F = BlambdaR @ KRR_inv @ BlambdaR.T \
    + BlambdaR @ KRR_inv @ KRP @ SPP_inv @ KPR @ KRR_inv @ BlambdaR.T \
    + BlambdaP @ SPP_inv @ BlambdaP.T \
    - BlambdaP @ SPP_inv @ KPR @ KRR_inv @ BlambdaR.T \
    - BlambdaR @ KRR_inv @ KRP @ SPP_inv @ BlambdaP.T

In [12]:
# Solve lambda linear system
lambda_ = np.linalg.solve(F, dH)

In [13]:
# Compute u
# uP
uP = SPP_inv @ fPH \
    + SPP_inv @ KPR @ KRR_inv @ BlambdaR.T @ lambda_ \
    - SPP_inv @ BlambdaP.T @ lambda_

# uR
uR = KRR_inv @ fR \
    + KRR_inv @ KRP @ SPP_inv @ KPR @ KRR_inv @ fR \
    - KRR_inv @ KRP @ SPP_inv @ fP \
    - KRR_inv @ KRP @ SPP_inv @ KPR @ KRR_inv @ BlambdaR.T @ lambda_ \
    + KRR_inv @ KRP @ SPP_inv @ BlambdaP.T @ lambda_ \
    - KRR_inv @ BlambdaR.T @ lambda_